In [2]:
import pickle
import os

In [3]:
with open('dic.pkl', 'rb') as f:
    dic = pickle.load(f)

In [4]:
lst = os.listdir('CompanyData')
orig = list(dic.keys())
chosen_list = []
delete_list = ['FALC', 'IDT', 'PRCP', 'BBOX', 'EVOL', 'CTS', 'FDC', 'NIHD', 'BRKS', 'KODK', 'OSPN', 'AEIS', 'LGL', 'QADA', 'PCMI']
for k in orig:
    if '{}output.xls'.format(k) not in lst or k in delete_list:
        del dic[k]
    else:
        chosen_list.append(k)
print(chosen_list)
print(len(chosen_list))
with open('chosen_list.txt', 'w') as f:
    for item in chosen_list:
        f.write("%s\n" % item)
with open('dic.pkl', 'wb') as f:
    pickle.dump(dic, f)

['INVE', 'PEGA', 'CASS', 'COHU', 'BDR', 'HMNY', 'BELFB', 'CDNS', 'KLAC', 'CY', 'XLNX', 'ZOOM', 'TACT', 'WGNR', 'NVDA', 'CMTL', 'AXE', 'TTWO', 'IT', 'INTC', 'ANEN', 'AMSWA', 'SYMC', 'AMKR', 'SPDC', 'CSCO', 'TLAB', 'SHEN', 'XRX', 'BHE', 'INTT', 'INAP', 'FTR', 'CRAY', 'SMIT', 'GVP', 'OLED', 'CYBE', 'MTSC', 'MCHP', 'ESCC', 'CTL', 'APH', 'LFUS', 'RMTR', 'VSAT', 'LSCC', 'ON', 'TDS', 'PAR', 'AAPL', 'PAYX', 'AXTI', 'EGOV', 'MMS', 'CELL', 'INTU', 'MSI', 'TRT', 'MSFT', 'QUIK', 'INOD', 'DGII', 'CSGS', 'DAIO', 'DDD', 'ASYS', 'WTT', 'EA', 'ITI', 'STCN', 'DIOD', 'NSYS', 'PLT', 'CYMI', 'MU', 'ZIXI', 'KVHI', 'NWK', 'DRCO', 'FICO', 'MSTR', 'IBM', 'CSPI', 'FSII', 'TYL', 'TXN', 'ADBE', 'HLIT', 'ALOT', 'CBB', 'PKE', 'SUPX', 'ADI', 'FARO', 'FFIV', 'EEFT', 'FLEX', 'VECO', 'MKSI', 'AWRE', 'CACI', 'FLIR', 'IAC', 'TSS', 'CIEN', 'TESS', 'MXWL', 'PRST', 'EBIX', 'IEC', 'VSH', 'ASUR', 'ZBRA', 'PCTI', 'TER', 'ANSS', 'TCX', 'EGHT', 'AMD', 'ACCL', 'WDC', 'OSIS', 'CREE', 'PFSW', 'ATEA', 'KLIC', 'NCR', 'CNXN', 'AVX', '

In [5]:
n_of_comp = len(chosen_list)

In [14]:
import numpy as np
def get_data(st, ed):
    X, Y = [], []
    companies = list(dic.keys())
    for comp in companies[st : ed]:
        for i in range(len(dic[comp]['sentiment_list']) - 1):
            X.append(dic[comp]['sentiment_list'][i])
            Y.append([dic[comp]['return_rate'][i]])
    X, Y = np.array(X), np.array(Y)
    return X, Y

In [15]:
from sklearn.linear_model import Ridge, LinearRegression
def train_linear_regression(X, Y, Xall, Yall):
    X = (X - np.mean(Xall, axis = 0)) / np.std(Xall, axis = 0)
    clf = Ridge(alpha = 1)
    clf.fit(X, Y)
    #print('training:')
    #print('L2 error:\t', np.sum((clf.predict(X) - Y) ** 2) / Y.shape[0])
    #print('score:\t', clf.score(X, Y))
    #print('validation:')
    #print('L2 error:\t', np.sum((clf.predict(Xval) - Yval) ** 2) / Yval.shape[0])
    #print('score:\t', clf.score(Xval, Yval))
    r = np.corrcoef(clf.predict(X).reshape(-1), Y.reshape(-1))
    rall = np.corrcoef(clf.predict(Xall).reshape(-1), Yall.reshape(-1))
    #print('\nCorrelated Coefficient:\n', r[0][1], rall[0][1])
    pred = []
    lenx = Xall.shape[0]
    for i in range(n_of_comp):
        Xind = (Xall[lenx //  n_of_comp * i: lenx // n_of_comp * (i + 1)] - np.mean(Xall, axis = 0)) / np.std(Xall, axis = 0)
        pred.append(clf.predict(Xind).reshape(-1))
    return np.array(pred)

In [10]:
import numpy as np
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
def train_dnn(X, Y, Xall, Yall):
    X = (X - np.mean(Xall, axis = 0)) / np.std(Xall, axis = 0)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    model = Sequential()
    model.add(Dense(128, input_shape = (X.shape[1], 1)))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(128, activation = 'sigmoid'))
    model.add(BatchNormalization())
    model.add(Dense(1))
    model.compile(loss = 'mse', optimizer = Adam(), metrics = [])
    model.fit(X, Y, batch_size = 32, epochs = 20)
    pred = []
    for i in range(n_of_comp):
        Xind = (Xall[lenx //  n_of_comp * i: lenx // n_of_comp * (i + 1)] - np.mean(Xall, axis = 0)) / np.std(Xall, axis = 0)
        tmp = model.predict(Xind.reshape((Xind.shape[0], Xind.shape[1], 1))).reshape(-1)
        pred.append(tmp)
    return np.array(pred)

In [16]:
Xall, Yall = get_data(0, n_of_comp)
lenx = Xall.shape[0]
K = 5
pred = np.zeros((n_of_comp, 29))
for i in range(K):
    X, Y = get_data(n_of_comp // K * i, n_of_comp // K * (i + 1))
    pred += train_linear_regression(X, Y, Xall, Yall)
pred /= K
print(pred.shape)
r = [np.corrcoef(pred[i], Yall[lenx // n_of_comp * i: lenx // n_of_comp * (i + 1)].reshape(-1))[0][1] for i in range(n_of_comp)]
companies = list(dic.keys())
rank = [(r[i], companies[i]) for i in range(n_of_comp)]
rank = sorted(rank, reverse = True)
print(np.mean(np.array(r), axis = None))
rank

(135, 29)
0.05299413302613838


[(0.567836462571689, 'DRCO'),
 (0.5317574650305604, 'CTSH'),
 (0.5206645461725282, 'CY'),
 (0.48420819349997885, 'FTR'),
 (0.48185183537680976, 'SUPX'),
 (0.4739617681191765, 'INTT'),
 (0.4580582957089906, 'AMD'),
 (0.4541097210500637, 'KVHI'),
 (0.4454630217625652, 'IEC'),
 (0.4422430027199356, 'ANEN'),
 (0.43223318099685276, 'TER'),
 (0.3873276574534337, 'CASS'),
 (0.3800596362551532, 'IAC'),
 (0.3629525784536365, 'MSFT'),
 (0.3206213962749377, 'TDS'),
 (0.3190300151143602, 'PFSW'),
 (0.3152850270790419, 'EEFT'),
 (0.3083065118784467, 'TESS'),
 (0.29918051285483044, 'CSGS'),
 (0.29702943149381733, 'CSCO'),
 (0.29552832458810424, 'TXN'),
 (0.2819598530733538, 'MTSC'),
 (0.2713859802156315, 'HLIT'),
 (0.2668004696158329, 'EGOV'),
 (0.2538841218998099, 'WTT'),
 (0.2480557890052429, 'MSI'),
 (0.2456516235571589, 'CBB'),
 (0.2446809626746957, 'MSTR'),
 (0.23630993514322982, 'AMKR'),
 (0.22461527829350708, 'FARO'),
 (0.22431781081882324, 'INVE'),
 (0.22339736421477752, 'ADBE'),
 (0.2222733

In [13]:
r = np.load('dnn_r.npy')
companies = list(dic.keys())
rank = [(r[i], companies[i]) for i in range(n_of_comp)]
rank = sorted(rank, reverse = True)
print(np.mean(r, axis = None))
rank

0.04731170128553646


[(0.512742234348051, 'FEIM'),
 (0.4649327262384743, 'TDS'),
 (0.42689711970418215, 'ITI'),
 (0.41175644825821633, 'ANEN'),
 (0.4082962240357552, 'FICO'),
 (0.35760322119445986, 'PLT'),
 (0.35004778167903416, 'XLNX'),
 (0.3491599150705975, 'IEC'),
 (0.3418026413108292, 'CACI'),
 (0.3312735845663457, 'BHE'),
 (0.3292303552390127, 'ON'),
 (0.3265506722882933, 'FLIR'),
 (0.3077839624724794, 'TRMB'),
 (0.3068480357798244, 'AAPL'),
 (0.302918373931888, 'CSGS'),
 (0.29743190899625727, 'DAIO'),
 (0.2876340309750617, 'MMS'),
 (0.273730696876465, 'CBB'),
 (0.2636983586535271, 'OLED'),
 (0.2579877089247843, 'TCX'),
 (0.2577999887574404, 'BELFB'),
 (0.2571000544861236, 'ALOT'),
 (0.2569972157411634, 'AXE'),
 (0.2500686343635304, 'TYL'),
 (0.2475504618442468, 'PAR'),
 (0.243059429149987, 'ZIXI'),
 (0.24145427373105144, 'CSCO'),
 (0.22860297565141302, 'VIAV'),
 (0.22697214796434173, 'COHU'),
 (0.2267246181440668, 'NVDA'),
 (0.2255260343575215, 'SMIT'),
 (0.2206054475487681, 'ASYS'),
 (0.214944209154